In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
import re
import warnings
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Functions

In [2]:
def calculate_round_winner(cs2, max_round):
    for round_number in range(1, max_round + 1):
        conditions = []
        choices = ['Favorite', 'Underdog', 'Favorite', 'Underdog']

        # Define conditions based on round number
        if round_number == 1:
            conditions = [
                (cs2['Moneyline A'] < cs2['Moneyline B']) & (cs2['R1 Score (A)'] > cs2['R1 Score (B)']),
                (cs2['Moneyline A'] > cs2['Moneyline B']) & (cs2['R1 Score (A)'] > cs2['R1 Score (B)']),
                (cs2['Moneyline B'] < cs2['Moneyline A']) & (cs2['R1 Score (B)'] > cs2['R1 Score (A)']),
                (cs2['Moneyline B'] > cs2['Moneyline A']) & (cs2['R1 Score (B)'] > cs2['R1 Score (A)'])
            ]
        elif round_number == 2:
            conditions = [
                (cs2['Moneyline A'] < cs2['Moneyline B']) & (cs2['R2 Score (A)'] > cs2['R2 Score (B)']),
                (cs2['Moneyline A'] > cs2['Moneyline B']) & (cs2['R2 Score (A)'] > cs2['R2 Score (B)']),
                (cs2['Moneyline B'] < cs2['Moneyline A']) & (cs2['R2 Score (B)'] > cs2['R2 Score (A)']),
                (cs2['Moneyline B'] > cs2['Moneyline A']) & (cs2['R2 Score (B)'] > cs2['R2 Score (A)'])
            ]
        elif round_number == 3:
            # Check if the 'R3 Score' columns exist
            if 'R3 Score (A)' in cs2.columns and 'R3 Score (B)' in cs2.columns:
                conditions = [
                    (cs2['Moneyline A'] < cs2['Moneyline B']) & (cs2['R3 Score (A)'] > cs2['R3 Score (B)']),
                    (cs2['Moneyline A'] > cs2['Moneyline B']) & (cs2['R3 Score (A)'] > cs2['R3 Score (B)']),
                    (cs2['Moneyline B'] < cs2['Moneyline A']) & (cs2['R3 Score (B)'] > cs2['R3 Score (A)']),
                    (cs2['Moneyline B'] > cs2['Moneyline A']) & (cs2['R3 Score (B)'] > cs2['R3 Score (A)'])
                ]

        # Apply conditions and choices
        if conditions:
            cs2[f'Round {round_number} Winner'] = np.select(conditions, choices, default='')
        else:
            cs2[f'Round {round_number} Winner'] = ''

    return cs2

# CS Data

In [3]:
fp = '/content/drive/My Drive/Sports Betting/CS2 - Data.xlsx'

cs2 = pd.read_excel(fp)

In [4]:
cs2

,Date,Type,League,Best Of,Stars,Team A,Team B,Odds Source,Moneyline A,Moneyline B,...,R3 Score (B),R3 Map,R4 Score (A),R4 Score (B),R4 Map,R5 Score (A),R5 Score (B),R5 Map,Score Count (A),Score Count (B)
0,2023-10-16,Intl. LAN,IEM Sydney 2023,3,0.0,Apeks,VERTEX,Oddsportal,1.20,4.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-10-16,Intl. LAN,IEM Sydney 2023,3,1.0,Monte,Grayhound,Oddsportal,1.32,3.19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-10-16,Intl. LAN,IEM Sydney 2023,3,3.0,Vitality,FaZe,Oddsportal,1.55,2.40,...,13.0,Nuke,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-10-16,Intl. LAN,IEM Sydney 2023,3,1.0,BetBoom,GamerLegion,Oddsportal,1.65,2.18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-10-16,Intl. LAN,IEM Sydney 2023,3,2.0,Complexity,G2,Oddsportal,2.64,1.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,2024-05-21,Online,CCT Global Finals 2024,3,1.0,Aurora,Astralis,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499,2024-05-21,Online,CCT Global Finals 2024,3,1.0,Liquid,ENCE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
500,2024-05-22,Online,CCT Global Finals 2024,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
501,2024-05-22,Online,CCT Global Finals 2024,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
cs2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             503 non-null    datetime64[ns]
 1   Type             503 non-null    object        
 2   League           503 non-null    object        
 3   Best Of          503 non-null    int64         
 4   Stars            500 non-null    float64       
 5   Team A           500 non-null    object        
 6   Team B           500 non-null    object        
 7   Odds Source      479 non-null    object        
 8   Moneyline A      486 non-null    float64       
 9   Moneyline B      486 non-null    float64       
 10  R1 Score (A)     498 non-null    float64       
 11  R1 Score (B)     498 non-null    float64       
 12  R1 Map           495 non-null    object        
 13  R2 Score (A)     498 non-null    float64       
 14  R2 Score (B)     498 non-null    float64  

In [6]:
cs2['Score Count (A)'] = (
    np.where((cs2['Best Of'] == 3) & (cs2['R1 Score (A)'] > cs2['R1 Score (B)']), 1, 0) +
    np.where((cs2['Best Of'] == 3) & (cs2['R2 Score (A)'] > cs2['R2 Score (B)']), 1, 0) +
    np.where((cs2['Best Of'] == 3) & (cs2['R3 Score (A)'] != 0) & (cs2['R3 Score (A)'] > cs2['R3 Score (B)']), 1, 0) +
    np.where((cs2['Best Of'] == 5) & (cs2['R1 Score (A)'] > cs2['R1 Score (B)']), 1, 0) +
    np.where((cs2['Best Of'] == 5) & (cs2['R2 Score (A)'] > cs2['R2 Score (B)']), 1, 0) +
    np.where((cs2['Best Of'] == 5) & (cs2['R3 Score (A)'] > cs2['R3 Score (B)']), 1, 0) +
    np.where((cs2['Best Of'] == 5) & (cs2['R4 Score (A)'] > cs2['R4 Score (B)']), 1, 0) +
    np.where((cs2['Best Of'] == 5) & (cs2['R5 Score (A)'] != 0) & (cs2['R5 Score (A)'] > cs2['R5 Score (B)']), 1, 0)
)

In [7]:
cs2['Score Count (B)'] = (
    np.where((cs2['Best Of'] == 3) & (cs2['R1 Score (B)'] > cs2['R1 Score (A)']), 1, 0) +
    np.where((cs2['Best Of'] == 3) & (cs2['R2 Score (B)'] > cs2['R2 Score (A)']), 1, 0) +
    np.where((cs2['Best Of'] == 3) & (cs2['R3 Score (A)'] != 0) & (cs2['R3 Score (B)'] > cs2['R3 Score (A)']), 1, 0) +
    np.where((cs2['Best Of'] == 5) & (cs2['R1 Score (B)'] > cs2['R1 Score (A)']), 1, 0) +
    np.where((cs2['Best Of'] == 5) & (cs2['R2 Score (B)'] > cs2['R2 Score (A)']), 1, 0) +
    np.where((cs2['Best Of'] == 5) & (cs2['R3 Score (B)'] > cs2['R3 Score (A)']), 1, 0) +
    np.where((cs2['Best Of'] == 5) & (cs2['R4 Score (B)'] > cs2['R4 Score (A)']), 1, 0) +
    np.where((cs2['Best Of'] == 5) & (cs2['R5 Score (A)'] != 0) & (cs2['R5 Score (B)'] > cs2['R5 Score (A)']), 1, 0)
)


In [8]:
cs2

,Date,Type,League,Best Of,Stars,Team A,Team B,Odds Source,Moneyline A,Moneyline B,...,R3 Score (B),R3 Map,R4 Score (A),R4 Score (B),R4 Map,R5 Score (A),R5 Score (B),R5 Map,Score Count (A),Score Count (B)
0,2023-10-16,Intl. LAN,IEM Sydney 2023,3,0.0,Apeks,VERTEX,Oddsportal,1.20,4.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0
1,2023-10-16,Intl. LAN,IEM Sydney 2023,3,1.0,Monte,Grayhound,Oddsportal,1.32,3.19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0
2,2023-10-16,Intl. LAN,IEM Sydney 2023,3,3.0,Vitality,FaZe,Oddsportal,1.55,2.40,...,13.0,Nuke,NaN,NaN,NaN,NaN,NaN,NaN,1,2
3,2023-10-16,Intl. LAN,IEM Sydney 2023,3,1.0,BetBoom,GamerLegion,Oddsportal,1.65,2.18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0
4,2023-10-16,Intl. LAN,IEM Sydney 2023,3,2.0,Complexity,G2,Oddsportal,2.64,1.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,2024-05-21,Online,CCT Global Finals 2024,3,1.0,Aurora,Astralis,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
499,2024-05-21,Online,CCT Global Finals 2024,3,1.0,Liquid,ENCE,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
500,2024-05-22,Online,CCT Global Finals 2024,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
501,2024-05-22,Online,CCT Global Finals 2024,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [9]:
cs2['Number of Rounds'] = cs2['Score Count (A)'] + cs2['Score Count (B)']

In [10]:
# Initialize the Moneyline Result column
cs2['Moneyline Result'] = np.where(cs2['Score Count (A)'] > cs2['Score Count (B)'],
                                   cs2['Moneyline A'],
                                   np.where(cs2['Score Count (B)'] > cs2['Score Count (A)'],
                                            cs2['Moneyline B'],
                                            np.nan))

In [11]:
conditions = [
    (cs2['Moneyline Result'] == cs2['Moneyline A']) & (cs2['Moneyline A'] < cs2['Moneyline B']),
    (cs2['Moneyline Result'] == cs2['Moneyline B']) & (cs2['Moneyline B'] < cs2['Moneyline A']),
    (cs2['Moneyline Result'] == cs2['Moneyline B']) & (cs2['Moneyline B'] > cs2['Moneyline A']),
    (cs2['Moneyline Result'] == cs2['Moneyline A']) & (cs2['Moneyline A'] > cs2['Moneyline B'])
]

# Results
results = ['Favorite', 'Favorite', 'Underdog', 'Underdog']

# Add Winner column based on Moneyline comparison
cs2['Winner'] = np.select(conditions, results, default=np.nan)

In [12]:
cs2['Winning Team'] = np.where(
    cs2['Moneyline Result'] == cs2['Moneyline A'],
    cs2['Team A'],
    np.where(
        cs2['Moneyline Result'] == cs2['Moneyline B'],
        cs2['Team B'],
        np.nan
    )
)

In [13]:
cs2 = cs2[cs2['Best Of'] == 3]

In [14]:
columns_to_drop = ['R4 Score (A)', 'R4 Score (B)', 'R5 Score (A)', 'R5 Score (B)']

cs2 = cs2.drop(columns=columns_to_drop, axis=1)

In [15]:
cs2.head()

,Date,Type,League,Best Of,Stars,Team A,Team B,Odds Source,Moneyline A,Moneyline B,...,R3 Score (B),R3 Map,R4 Map,R5 Map,Score Count (A),Score Count (B),Number of Rounds,Moneyline Result,Winner,Winning Team
0,2023-10-16,Intl. LAN,IEM Sydney 2023,3,0.0,Apeks,VERTEX,Oddsportal,1.20,4.01,...,NaN,NaN,NaN,NaN,2,0,2,1.20,Favorite,Apeks
1,2023-10-16,Intl. LAN,IEM Sydney 2023,3,1.0,Monte,Grayhound,Oddsportal,1.32,3.19,...,NaN,NaN,NaN,NaN,2,0,2,1.32,Favorite,Monte
2,2023-10-16,Intl. LAN,IEM Sydney 2023,3,3.0,Vitality,FaZe,Oddsportal,1.55,2.40,...,13.0,Nuke,NaN,NaN,1,2,3,2.40,Underdog,FaZe
3,2023-10-16,Intl. LAN,IEM Sydney 2023,3,1.0,BetBoom,GamerLegion,Oddsportal,1.65,2.18,...,NaN,NaN,NaN,NaN,2,0,2,1.65,Favorite,BetBoom
4,2023-10-16,Intl. LAN,IEM Sydney 2023,3,2.0,Complexity,G2,Oddsportal,2.64,1.45,...,NaN,NaN,NaN,NaN,0,2,2,1.45,Favorite,G2


### Round Winners

In [16]:
cs2 = calculate_round_winner(cs2, 3)
cs2

,Date,Type,League,Best Of,Stars,Team A,Team B,Odds Source,Moneyline A,Moneyline B,...,R5 Map,Score Count (A),Score Count (B),Number of Rounds,Moneyline Result,Winner,Winning Team,Round 1 Winner,Round 2 Winner,Round 3 Winner
0,2023-10-16,Intl. LAN,IEM Sydney 2023,3,0.0,Apeks,VERTEX,Oddsportal,1.20,4.01,...,NaN,2,0,2,1.20,Favorite,Apeks,Favorite,Favorite,
1,2023-10-16,Intl. LAN,IEM Sydney 2023,3,1.0,Monte,Grayhound,Oddsportal,1.32,3.19,...,NaN,2,0,2,1.32,Favorite,Monte,Favorite,Favorite,
2,2023-10-16,Intl. LAN,IEM Sydney 2023,3,3.0,Vitality,FaZe,Oddsportal,1.55,2.40,...,NaN,1,2,3,2.40,Underdog,FaZe,Favorite,Underdog,Underdog
3,2023-10-16,Intl. LAN,IEM Sydney 2023,3,1.0,BetBoom,GamerLegion,Oddsportal,1.65,2.18,...,NaN,2,0,2,1.65,Favorite,BetBoom,Favorite,Favorite,
4,2023-10-16,Intl. LAN,IEM Sydney 2023,3,2.0,Complexity,G2,Oddsportal,2.64,1.45,...,NaN,0,2,2,1.45,Favorite,G2,Favorite,Favorite,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,2024-05-21,Online,CCT Global Finals 2024,3,1.0,Aurora,Astralis,NaN,NaN,NaN,...,NaN,0,0,0,NaN,nan,NaN,,,
499,2024-05-21,Online,CCT Global Finals 2024,3,1.0,Liquid,ENCE,NaN,NaN,NaN,...,NaN,0,0,0,NaN,nan,NaN,,,
500,2024-05-22,Online,CCT Global Finals 2024,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,0,NaN,nan,NaN,,,
501,2024-05-22,Online,CCT Global Finals 2024,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,0,NaN,nan,NaN,,,


In [17]:
cs2['Underdog Comeback'] = np.where(
    (cs2['Round 2 Winner'] == 'Underdog') & (cs2['Round 3 Winner'] == 'Underdog'),
    'Yes',
    'No'
)


In [18]:
cs2.head()

,Date,Type,League,Best Of,Stars,Team A,Team B,Odds Source,Moneyline A,Moneyline B,...,Score Count (A),Score Count (B),Number of Rounds,Moneyline Result,Winner,Winning Team,Round 1 Winner,Round 2 Winner,Round 3 Winner,Underdog Comeback
0,2023-10-16,Intl. LAN,IEM Sydney 2023,3,0.0,Apeks,VERTEX,Oddsportal,1.20,4.01,...,2,0,2,1.20,Favorite,Apeks,Favorite,Favorite,,No
1,2023-10-16,Intl. LAN,IEM Sydney 2023,3,1.0,Monte,Grayhound,Oddsportal,1.32,3.19,...,2,0,2,1.32,Favorite,Monte,Favorite,Favorite,,No
2,2023-10-16,Intl. LAN,IEM Sydney 2023,3,3.0,Vitality,FaZe,Oddsportal,1.55,2.40,...,1,2,3,2.40,Underdog,FaZe,Favorite,Underdog,Underdog,Yes
3,2023-10-16,Intl. LAN,IEM Sydney 2023,3,1.0,BetBoom,GamerLegion,Oddsportal,1.65,2.18,...,2,0,2,1.65,Favorite,BetBoom,Favorite,Favorite,,No
4,2023-10-16,Intl. LAN,IEM Sydney 2023,3,2.0,Complexity,G2,Oddsportal,2.64,1.45,...,0,2,2,1.45,Favorite,G2,Favorite,Favorite,,No


# Analysis

## Winner overall

In [19]:
ud_win_overall = cs2['Winner'].eq("Underdog").sum()
ud_win_overall

163

In [20]:
# Total matches
total_matches = cs2['Winner'].count()

In [21]:
ud_win_pct = ud_win_overall / total_matches
ud_win_pct

0.3286290322580645

## Filter for strategy (map 3)

In [22]:
cs2 = cs2[cs2['Number of Rounds'] == 3]
cs2

,Date,Type,League,Best Of,Stars,Team A,Team B,Odds Source,Moneyline A,Moneyline B,...,Score Count (A),Score Count (B),Number of Rounds,Moneyline Result,Winner,Winning Team,Round 1 Winner,Round 2 Winner,Round 3 Winner,Underdog Comeback
2,2023-10-16,Intl. LAN,IEM Sydney 2023,3,3.0,Vitality,FaZe,Oddsportal,1.55,2.40,...,1,2,3,2.40,Underdog,FaZe,Favorite,Underdog,Underdog,Yes
5,2023-10-16,Intl. LAN,IEM Sydney 2023,3,4.0,Natus Vincere,MOUZ,Oddsportal,4.36,1.21,...,1,2,3,1.21,Favorite,MOUZ,Favorite,Underdog,Favorite,No
6,2023-10-17,Intl. LAN,IEM Sydney 2023,3,2.0,ENCE,fnatic,Oddsportal,1.42,2.73,...,2,1,3,1.42,Favorite,ENCE,Underdog,Favorite,Favorite,No
7,2023-10-17,Intl. LAN,IEM Sydney 2023,3,2.0,Monte,fnatic,Oddsportal,1.92,1.82,...,2,1,3,1.92,Underdog,Monte,Favorite,Underdog,Underdog,Yes
9,2023-10-17,Intl. LAN,IEM Sydney 2023,3,2.0,Complexity,Cloud9,Oddsportal,2.41,1.53,...,2,1,3,2.41,Underdog,Complexity,Underdog,Favorite,Underdog,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,2024-05-19,Online,CCT Global Finals 2024,3,1.0,paiN,OG,NaN,1.76,1.98,...,2,1,3,1.76,Favorite,paiN,Underdog,Favorite,Favorite,No
493,2024-05-20,Online,CCT Global Finals 2024,3,2.0,MIBR,BetBoom,NaN,1.55,2.38,...,1,2,3,2.38,Underdog,BetBoom,Favorite,Underdog,Underdog,Yes
495,2024-05-20,Online,CCT Global Finals 2024,3,1.0,BIG,MIBR,NaN,2.01,1.74,...,1,2,3,1.74,Favorite,MIBR,Favorite,Underdog,Favorite,No
496,2024-05-21,Online,CCT Global Finals 2024,3,0.0,GamerLegion,AMKAL,NaN,NaN,NaN,...,2,1,3,NaN,nan,NaN,,,,No


In [23]:
cs2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 217 entries, 2 to 497
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Date               217 non-null    datetime64[ns]
 1   Type               217 non-null    object        
 2   League             217 non-null    object        
 3   Best Of            217 non-null    int64         
 4   Stars              217 non-null    float64       
 5   Team A             217 non-null    object        
 6   Team B             217 non-null    object        
 7   Odds Source        209 non-null    object        
 8   Moneyline A        212 non-null    float64       
 9   Moneyline B        212 non-null    float64       
 10  R1 Score (A)       217 non-null    float64       
 11  R1 Score (B)       217 non-null    float64       
 12  R1 Map             214 non-null    object        
 13  R2 Score (A)       217 non-null    float64       
 14  R2 Score (B)   

In [24]:
cs2['Round 3 Winner'].count()

217

In [25]:
# Round 3 Underdog Winner
r3_ud = cs2['Round 3 Winner'].eq("Underdog").sum()
r3_ud

90

In [26]:
# Round 3 Underdog Winner divided by Total 3 Round Matches
r3_ud_win = r3_ud / cs2['Round 3 Winner'].count()

In [27]:
r3_ud_win.round(5)

0.41475

In [28]:
underdog_comeback = ((cs2['Round 2 Winner'] == "Underdog") & (cs2['Round 3 Winner'] == "Underdog")).sum()
underdog_comeback

40

In [29]:
favorite_comeback = ((cs2['Round 2 Winner'] == "Favorite") & (cs2['Round 3 Winner'] == "Favorite")).sum()
favorite_comeback

60

In [30]:
ud_cb = underdog_comeback / cs2['Round 3 Winner'].count()
ud_cb

0.18433179723502305

In [31]:
fv_cb = favorite_comeback / cs2['Round 3 Winner'].count()
fv_cb

0.2764976958525346

# Results

In [32]:
print("Underdogs win", round((ud_win_pct)*100,5), "% of the time overall.")
print("Fair price =", round(1/ud_win_pct, 5),".")
print()
print("Underdogs win", round(r3_ud_win * 100, 5), "% of the time in map 3.")
print("Fair price =", round(1/r3_ud_win, 5),".")
print()
print("Underdogs comeback at", round(ud_cb * 100, 5), "% of the time.")
print("Fair price =", round(1/ud_cb, 5),".")

Underdogs win 32.8629 % of the time overall.
Fair price = 3.04294 .

Underdogs win 41.47465 % of the time in map 3.
Fair price = 2.41111 .

Underdogs comeback at 18.43318 % of the time.
Fair price = 5.425 .


In [33]:
print("Favorites win", round((1-(ud_win_pct))*100,5), "% of the time overall.")
print("Fair price =", round(1/(1- ud_win_pct), 5),".")
print()
print("Favorites win", round((1 - r3_ud_win) * 100, 5), "% of the time in map 3.")
print("Fair price =", round(1/(1 - r3_ud_win), 5),".")
print()
print("Favorites comeback at", round(fv_cb * 100, 5), "% of the time.")
print("Fair price =", round(1/fv_cb, 5),".")

Favorites win 67.1371 % of the time overall.
Fair price = 1.48949 .

Favorites win 58.52535 % of the time in map 3.
Fair price = 1.70866 .

Favorites comeback at 27.64977 % of the time.
Fair price = 3.61667 .
